In [14]:
from typing import List

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

YOUR_API_URL = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/df6ac0a24d53974e"


def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
    

## A simple chat bot

In [15]:
config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gpt-4o", parameters={"temperature": 0.0}),
        )

class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService, config: OrchestrationConfig):
        self.service = orchestration_service
        self.history: List[Message] = []
        self.config = config

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

service = OrchestrationService(api_url=YOUR_API_URL)
bot = ChatBot(orchestration_service=service, config=config)


In [16]:
print(bot.chat("Hello, how are you?"))

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [17]:
print(bot.chat("What's the weather like today?"))

I'm sorry, but I can't provide real-time weather updates. However, you can check the weather by using a weather app or website, or by asking a voice-activated assistant on your device. If you need help with anything else, feel free to ask!


In [18]:
print(bot.chat("Can you remember what I first asked you?"))

I don't have the ability to remember past interactions or previous questions. However, I'm here to help with any new questions or topics you have. How can I assist you today?


## Making our chat bot responsible
Adding content filtering and data masking by only changing the config

In [19]:
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, \
    ProfileEntity
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold


data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)

output_filter = AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                   violence=AzureThreshold.ALLOW_SAFE_LOW,
                                   self_harm=AzureThreshold.ALLOW_SAFE_LOW_MEDIUM,
                                   sexual=AzureThreshold.ALLOW_ALL)

config.data_masking = data_masking
config.output_filters = [output_filter]


Handling sensitive information

In [20]:
cv = load_text_file("data/cv.txt")
prompt = "Summarize the following CV in 10 sentences: {cv}".format(cv=cv)

print(prompt + "\n")

Summarize the following CV in 10 sentences: Patrick Morgan
+49 30 23125 123
patric.morgan@example.com

Highlights

Strategic and financial planning expert
Accurate forecasting
Process implementation
Staff leadership and development
Business performance improvement
Proficient in SAP, Excel VBA
Education
Master of Science: Finance - 2014
Harvard University, Boston

Bachelor of Science: Finance - 2011
Harvard University, Boston

Certifications
Certified Management Accountant

Summary
Skilled Financial Manager adept at increasing work process efficiency and profitability through functional and technical analysis.
Successful at advising large corporations, small businesses, and individual clients. Areas of expertise include asset allocation, investment strategy, and risk management.

Experience
Finance Manager - 09/2016 to 05/2018
M&K Group, York

Manage the modelling, planning, and execution of all financial processes.
Carry short and long-term custom comprehensive financial strategies to 

In [21]:
print(bot.chat(prompt))

The CV belongs to a financial expert with a strong background in strategic and financial planning, forecasting, and process implementation. They hold both a Master and Bachelor of Science in Finance from Harvard University, completed in 2014 and 2011, respectively. The individual is a Certified Management Accountant, showcasing their proficiency in financial management. They have experience advising large corporations, small businesses, and individual clients, with expertise in asset allocation, investment strategy, and risk management. Their professional experience includes roles as a Finance Manager at two organizations from 2013 to 2018. In these roles, they managed financial processes, developed financial strategies, and recommended revenue-generating alternatives. They also conducted advanced deal analysis and negotiated with potential investors. Additionally, they have skills in business performance improvement and staff leadership, and are proficient in Excel VBA.
